### Import Dependencies

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D, MaxPool2D, Dropout
from tensorflow.keras.losses import SparseCategoricalCrossentropy

import numpy as np
import matplotlib.pyplot as plt

### Load Data

In [ ]:
builder = tfds.builder('rock_paper_scissors')
info = builder.info

info

tfds.core.DatasetInfo(
    name='rock_paper_scissors',
    full_name='rock_paper_scissors/3.0.0',
    description="""
    Images of hands playing rock, paper, scissor game.
    """,
    homepage='http://laurencemoroney.com/rock-paper-scissors-dataset',
    data_dir='/root/tensorflow_datasets/rock_paper_scissors/3.0.0',
    file_format=tfrecord,
    download_size=219.53 MiB,
    dataset_size=219.23 MiB,
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=3),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=372, num_shards=1>,
        'train': <SplitInfo num_examples=2520, num_shards=2>,
    },
    citation="""@ONLINE {rps,
    author = "Laurence Moroney",
    title = "Rock, Paper, Scissors Dataset",
    month = "feb",
    year = "2019",
    url = "http://laurencemoroney.com/rock-paper-scissors-dataset"
    }""",

In [ ]:
all_train = tfds.load(name="rock_paper_scissors", split="train")
all_test = tfds.load(name="rock_paper_scissors", split="test")

In [ ]:
len(all_train), len(all_test)

(2520, 372)

### Data Processing

In [ ]:
# Convert tfds to np

train_images = np.array([item['image'].numpy()[:,:,0] for item in all_train])  # Only take one color channel since color is not required
train_labels = np.array([item['label'].numpy() for item in all_train])

test_images = np.array([item['image'].numpy()[:,:,0] for item in all_test])  # Only take one color channel since color is not required
test_labels = np.array([item['label'].numpy() for item in all_test])

In [ ]:
print(train_images.shape)
print(test_images.shape)

(2520, 300, 300)
(372, 300, 300)


In [ ]:
# Reshape the data so that the one color channel is recognized

train_images = train_images.reshape(2520, 300, 300, 1)
test_images = test_images.reshape(372, 300, 300, 1)

In [ ]:
train_images.dtype

dtype('uint8')

In [ ]:
# Convert the dataset to float so that range can be scaled from 0-255 to 0-1

train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

In [ ]:
# Rescale the image dataset

train_images /= 255
test_images /= 255


### First approach: ANN

In [ ]:
model = Sequential([
    Flatten(),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size=32)

Epoch 1/5
79/79 [==============================] - 4s 23ms/step - loss: 19.2124 - accuracy: 0.4254
Epoch 2/5
79/79 [==============================] - 2s 21ms/step - loss: 1.8144 - accuracy: 0.6393
Epoch 3/5
79/79 [==============================] - 2s 22ms/step - loss: 2.0567 - accuracy: 0.6611
Epoch 4/5
79/79 [==============================] - 2s 22ms/step - loss: 0.9032 - accuracy: 0.7806
Epoch 5/5
79/79 [==============================] - 2s 19ms/step - loss: 0.8522 - accuracy: 0.8036


In [ ]:
model.evaluate(test_images, test_labels)

12/12 [==============================] - 0s 10ms/step - loss: 1.6374 - accuracy: 0.5995


[1.6373717784881592, 0.5994623899459839]

In [ ]:
# Overfitting occured. Use CNN instead

### Second Approach: Simple CNN

In [ ]:
model = Sequential([
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(300,300,1)),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    Flatten(),
    Dense(3, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size=32)

Epoch 1/5
79/79 [==============================] - 26s 203ms/step - loss: 3.7192 - accuracy: 0.7024
Epoch 2/5
79/79 [==============================] - 11s 145ms/step - loss: 0.2716 - accuracy: 0.9524
Epoch 3/5
79/79 [==============================] - 11s 145ms/step - loss: 0.1063 - accuracy: 0.9857
Epoch 4/5
79/79 [==============================] - 11s 145ms/step - loss: 0.0296 - accuracy: 0.9968
Epoch 5/5
79/79 [==============================] - 11s 145ms/step - loss: 0.0085 - accuracy: 1.0000


In [ ]:
model.evaluate(test_images, test_labels)

12/12 [==============================] - 2s 188ms/step - loss: 1.0935 - accuracy: 0.5457


[1.0935419797897339, 0.5456989407539368]

In [ ]:
# Overfitting once again. Use a mix of different hyperparameters in the CNN

### Third Approach: Tuned CNN

In [ ]:
model = Sequential([
    AveragePooling2D(pool_size=(6,6), strides=3, input_shape=(300,300,1)),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=(300,300,1)),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Dropout(rate=0.5),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size=32)

Epoch 1/5
79/79 [==============================] - 7s 38ms/step - loss: 0.6505 - accuracy: 0.7516
Epoch 2/5
79/79 [==============================] - 2s 28ms/step - loss: 0.0719 - accuracy: 0.9837
Epoch 3/5
79/79 [==============================] - 2s 28ms/step - loss: 0.0149 - accuracy: 0.9980
Epoch 4/5
79/79 [==============================] - 2s 30ms/step - loss: 0.0063 - accuracy: 0.9992
Epoch 5/5
79/79 [==============================] - 2s 30ms/step - loss: 0.0074 - accuracy: 0.9984


In [ ]:
model.evaluate(test_images, test_labels)

12/12 [==============================] - 0s 30ms/step - loss: 0.9096 - accuracy: 0.6774


[0.9096390008926392, 0.6774193644523621]

### Hyperparameter Tuning

In [ ]:
# Use Keras Tuner
! pip install -U keras-tuner
import keras_tuner

In [ ]:
from keras_tuner.tuners import RandomSearch

def build_model(hp):
  model = Sequential()

  model.add(AveragePooling2D(pool_size=(6,6), strides=3, input_shape=(300,300,1)))

  for i in range(hp.Int("Conv2D Layers", min_value=0, max_value=3)):
        model.add(Conv2D(
            hp.Choice(f"layer_{i}_filters", [16,32,64]),
            kernel_size=(3,3),
            activation='relu'
            )
        )

  model.add(MaxPool2D(pool_size=(2,2), strides=2))
  model.add(Dropout(rate=0.5))
  model.add(Flatten())

  model.add(Dense(hp.Choice(
      "Dense layer",
      [64, 128, 256, 512, 1024]
      ),
      activation='relu'))

  model.add(Dense(3, activation='softmax'))
  model.compile(
    optimizer='adam',
    loss=SparseCategoricalCrossentropy(),
    metrics=['accuracy']
  )

  return model

tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    directory="./",
    project_name="new_tuner")

tuner.search(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels), batch_size=32)

Trial 10 Complete [00h 00m 08s]
val_loss: 1.0099250078201294

Best val_loss So Far: 0.6540169715881348
Total elapsed time: 00h 03m 16s


In [ ]:
best_model = tuner.get_best_models()[0]
best_model.evaluate(test_images, test_labels)

12/12 [==============================] - 0s 8ms/step - loss: 0.6540 - accuracy: 0.7715


[0.6540169715881348, 0.7715053558349609]

In [ ]:
# Check the parameters used in the best model
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 average_pooling2d (Average  (None, 99, 99, 1)         0         
 Pooling2D)                                                      
                                                                 
 conv2d (Conv2D)             (None, 97, 97, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 48, 48, 16)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 48, 48, 16)        0         
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 512)               1

In [ ]:
# Check the summary for each hyperparameter chosen
tuner.results_summary()

Results summary
Results in ./new_tuner
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 00 summary
Hyperparameters:
Conv2D Layers: 1
Dense layer: 512
layer_0_filters: 16
Score: 0.6540169715881348

Trial 06 summary
Hyperparameters:
Conv2D Layers: 1
Dense layer: 512
layer_0_filters: 64
layer_1_filters: 32
Score: 0.6906559467315674

Trial 08 summary
Hyperparameters:
Conv2D Layers: 1
Dense layer: 1024
layer_0_filters: 64
layer_1_filters: 32
Score: 0.7485895752906799

Trial 05 summary
Hyperparameters:
Conv2D Layers: 2
Dense layer: 128
layer_0_filters: 64
layer_1_filters: 32
Score: 0.7577353715896606

Trial 02 summary
Hyperparameters:
Conv2D Layers: 2
Dense layer: 1024
layer_0_filters: 64
layer_1_filters: 16
Score: 0.9102604985237122

Trial 01 summary
Hyperparameters:
Conv2D Layers: 0
Dense layer: 128
layer_0_filters: 32
Score: 0.9490323662757874

Trial 07 summary
Hyperparameters:
Conv2D Layers: 0
Dense layer: 1024
layer_0_filters: 16
layer_1_filters: 32
Score: 0.951

### Saving Model

In [ ]:
best_model.save("./my_model")

### Load Model

In [ ]:
from keras.models import load_model

loaded_model = load_model("./my_model")

In [ ]:
loaded_model.evaluate(test_images, test_labels)

12/12 [==============================] - 0s 8ms/step - loss: 0.6540 - accuracy: 0.7715


[0.6540169715881348, 0.7715053558349609]